In [16]:
import json
import random
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
# Unduh resource NLTK
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from google.colab import drive
drive.mount('/content/drive') # Sesuaikan path direktori
%cd /content/drive/My Drive/DM/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/DM


In [18]:
# Load dataset
with open("Assets/e-commerce.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Preprocessing helper
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [stemmer.stem(t) for t in tokens if t.isalpha() and t not in stop_words]
    return " ".join(tokens)

# Siapkan data training
X = []
y = []
responses = {}
for intent in data["intetns"]:
    tag = intent["tag"]
    for pattern in intent["patterns"]:
        X.append(preprocess(pattern))
        y.append(tag)
    responses[tag] = intent["responses"]
print(type(X))
# Cek Data
print(len(X), len(y))
for i in range(16):
    print(X[i], '\t', y[i])
print('...')
for i in range(4000, 4016):
    print(X[i], '\t', y[i])
print('...')
for i in range(len(X)-1, len(X)-21, -1):
    print(X[i], '\t', y[i])

<class 'list'>
8026 8026
would possibl cancel order made 	 cancel_order
cancel order 	 cancel_order
need assist cancel last order made 	 cancel_order
problem cancel order made 	 cancel_order
know cancel order made 	 cancel_order
help cancel order made 	 cancel_order
would like know order cancel 	 cancel_order
could help cancel order 	 cancel_order
know cancel order made 	 cancel_order
help cancel last order 	 cancel_order
know cancel last order made 	 cancel_order
need assist cancel order made 	 cancel_order
inform cancel order 	 cancel_order
would like cancel order 	 cancel_order
need assist cancel order 	 cancel_order
need assist cancel order made 	 cancel_order
...
want help check option deliveri offer 	 delivery_options
would like know deliveri option 	 delivery_options
help check deliveri option 	 delivery_options
check deliveri option choos 	 delivery_options
want inform deliveri option 	 delivery_options
could show deliveri option offer 	 delivery_options
could show option deliv

In [19]:
# Coba count vectorizer
import pandas as pd

print(type(X))
vectorizer = CountVectorizer()
text_vect = vectorizer.fit_transform(X)
vocabs = (vectorizer.get_feature_names_out())
print(set(y))
pd.DataFrame(text_vect.toarray(), columns=vocabs)


<class 'list'>
{'recover_password', 'check_refund_policy', 'contact_customer_service', 'get_refund', 'review', 'delete_account', 'set_up_shipping_address', 'delivery_options', 'change_order', 'get_invoice', 'track_order', 'payment_issue', 'registration_problems', 'track_refund', 'cancel_order', 'change_shipping_address', 'place_order', 'complaint', 'check_cancellation_fee', 'switch_account', 'create_account', 'delivery_period', 'contact_human_agent', 'check_invoice', 'check_payment_methods', 'edit_account', 'newsletter_subscription'}


,abuot,accdss,accept,access,accesw,accont,acconut,accouint,accounmt,accounnt,...,wit,withdraw,witth,word,work,would,wrong,ya,yet,youur
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8021,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8022,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8023,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8024,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Buat pipeline: CountVectorizer + MNB
model = Pipeline([
    ("vectorizer", CountVectorizer()),
    ("classifier", MultinomialNB())
])

print(model)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', MultinomialNB())])


In [22]:
import numpy as np

# Train model
model.fit(X, y)

def chat():
    print("Chatbot siap! (Ketik 'keluar' untuk berhenti)\n")
    while True:
        user_input = input("Kamu: ")
        if user_input.lower() == "keluar":
            print("Bot: Sampai jumpa!")
            break

        preprocessed = preprocess(user_input)
        proba = model.predict_proba([preprocessed])[0]
        max_proba = np.max(proba)

        if max_proba < 0.2:
            print("Sorry I’m don’t understand")
        else:
            predicted_intent = model.classes_[np.argmax(proba)]
            response = random.choice(responses[predicted_intent])
            print("Bot:", response)

# Jalankan chatbot
if __name__ == "__main__":
    chat()


Chatbot siap! (Ketik 'keluar' untuk berhenti)

Kamu: I want to cancel my order
Bot: No problem. I can help you with order cancellation. What's the order number?
Kamu: Can you help cancel my last purchase?
Bot: To cancel an order, I'll need some details. Could you provide your order ID or the email address used for the purchase?
Kamu: Can you check the status of my refund?
Bot: To find out if your refund has been processed, please share your original order details. I'll get you an update.
Kamu: I want to know where my refund is.
Bot: You can initiate a refund request through your account's purchase history or by contacting our support team directly. What is your order ID?
Kamu: What delivery options do you offer?
Bot: Yes, I can tell you about our delivery options. We have several choices available depending on your urgency and location.
Kamu: “Do you have express shipping?”
Bot: I can help you add a new delivery address to your account. Please ensure all fields like street, city, and p

In [23]:
import pickle
with open("model_chatbot.pkl", "wb") as model_file:
    pickle.dump(model, model_file)
